In [101]:
import pandas as pd 
import numpy as np
import sklearn
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler

post_data = pd.read_csv('/workspaces/Crowdfunding-Social-Media-Drivers/Data/Original_dataset/processed_data.csv')

targets = ['likes', 'shares', 'comments', 'positive_reactions', 'negative_reactions']
features = [x for x in post_data.columns if x not in targets]


features = post_data[features]
targets = post_data[targets]



In [102]:
def pre_process(x):
    scalable_features = ['likes_at_posting', 'followers_at_posting', 'post_views']
    noticable_entities = ['ORG', 'PERSON', 'DATE',
                          'CARDINAL', 'GPE', 'PRODUCT', 
                          'WORK_OF_ART','ORDINAL', 'MONEY',
                          'TIME', 'NORP']
    scaler = RobustScaler()
    x.loc[:,scalable_features] = scaler.fit_transform(x.loc[:,scalable_features])

    x = pd.concat([x, pd.get_dummies(x.type, prefix='type', drop_first=True).astype(int)], axis=1).drop('type', axis=1)
    x = pd.concat([x, pd.get_dummies(x.page_name, prefix='page_name').astype(int)], axis=1).drop('page_name', axis=1)
    for entity in noticable_entities:
        x[f'entity_{entity}'] = x.entities_identified.fillna('None').str.split(',').apply(lambda entity_list: entity in entity_list).astype(int)
    
    x = x.drop('entities_identified', axis=1)
    return x


def process_targets(y):
    y = y.copy()
    y = np.log(y+1)
    return y

def decode_targets(y):
    y = y.copy()
    y = np.exp(y)-1
    return y

In [103]:
processed_features = pre_process(features)
processed_targets = process_targets(targets)

In [104]:
x_train, x_test, y_train, y_test = train_test_split(processed_features, processed_targets, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

y_test = decode_targets(y_test)

In [108]:
#xgb training
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, max_depth=5, random_state=42)
xgb_model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=150, verbose=False)
xgb_pred = xgb_model.predict(x_test)
xgb_pred = decode_targets(xgb_pred)

print('XGB')
print('RMSE: ', np.sqrt(mean_squared_error(y_test, xgb_pred)))
print('R2: ', r2_score(y_test, xgb_pred))


/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGB
RMSE:  29.11420015874544
R2:  0.49737119692011433
